# Task 2
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [36]:
import numpy as np
import pandas as pd
import sklearn
# Add any other imports you need here

# Data Loading
TODO: Perform data preprocessing, imputation and extract X_train, y_train and X_test
(and potentially change initialization of variables to accomodate how you deal with non-numeric data)

In [102]:
"""
This loads the training and test data, preprocesses it, removes the NaN
values and interpolates the missing data using imputation

Parameters
----------
Compute
----------
X_train: matrix of floats, training input with features
y_train: array of floats, training output with labels
X_test: matrix of floats: dim = (100, ?), test input with features
"""
# Load training data
import sklearn.impute


train_df = pd.read_csv("train.csv")
    
# print("Training data:")
# print("Shape:", train_df.shape)
# print(train_df.head(4))
# print('\n')
    
# Load test data
test_df = pd.read_csv("test.csv")

# print("Test data:")
# print(test_df.shape)
# print(test_df.head(2))

# Dummy initialization of the X_train, X_test and y_train   
# TODO: Depending on how you deal with the non-numeric data, you may want to 
# modify/ignore the initialization of these variables   
X_train = np.zeros_like(train_df.drop(['price_CHF'],axis=1))
y_train = np.zeros_like(train_df['price_CHF'])
X_test = np.zeros_like(test_df)

# TODO: Perform data preprocessing, imputation and extract X_train, y_train and X_test

train_df_CHF = train_df.dropna(subset=['price_CHF']) # Drop all rows with nan entries in "price_CHF"
X_train = train_df_CHF.drop(['price_CHF'],axis=1)
y_train = train_df_CHF['price_CHF']
X_test = test_df

spring_df = train_df_CHF[train_df_CHF['season'] == 'spring']
summer_df = train_df_CHF[train_df_CHF['season'] == 'summer']
autumn_df = train_df_CHF[train_df_CHF['season'] == 'autumn']
winter_df = train_df_CHF[train_df_CHF['season'] == 'winter']

seasons_array = [spring_df, summer_df, autumn_df, winter_df]

for season_df in seasons_array:
    season_df_filled = pd.DataFrame(np.nan, index = season_df.index, columns = season_df.columns)
    season_df_filled.iloc[0:] = season_df.iloc[0]
    for i in range(1,11) :
        season_col_filled = season_df.iloc[:,i].fillna(np.mean(season_df.iloc[:,i]))
        print(np.mean(season_df.iloc[:,i]))
        season_df_filled.iloc[:, i] = season_col_filled
    print(season_df_filled)

assert (X_train.shape[1] == X_test.shape[1]) and (X_train.shape[0] == y_train.shape[0]) and (X_test.shape[0] == 100), "Invalid data shape"

-1.1311709251731938
1.0331678467272591
-1.181627948038101
-1.1120672158764764
-2.7106413355158687
-2.09448427614711
-1.666080102783169
-2.465162091497691
-0.6585457306773845
-1.1841483788541176
     season  price_AUS  price_CHF  price_CZE  price_GER  price_ESP  price_FRA  \
4    spring  -1.131171  -5.171675  -2.681391  -1.630410  -3.615413  -0.470111   
12   spring  -1.131171   1.952635   2.395004   2.684656  -1.338211  -2.094484   
16   spring   2.291247   0.978762   2.611865  -1.112067  -0.154121   1.734737   
20   spring  -1.131171  -3.089666   2.350411   1.813438   0.495253   1.570561   
24   spring  -1.131171  -1.815609   2.402741   1.543463   0.675973   1.533164   
..      ...        ...        ...        ...        ...        ...        ...   
880  spring  -1.131171  -1.070035  -1.181628  -3.533170  -1.528496  -3.733332   
884  spring  -1.131171   2.077429  -2.109459  -1.112067  -1.753443  -3.644403   
888  spring  -1.671694   4.756591  -1.835861  -2.572227  -2.710641  -2.094484

/var/folders/8x/k_nyp_s16d10yl_nkhlgg7cc0000gn/T/ipykernel_70200/309712317.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'spring' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  season_df_filled.iloc[0:] = season_df.iloc[0]
/var/folders/8x/k_nyp_s16d10yl_nkhlgg7cc0000gn/T/ipykernel_70200/309712317.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'summer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  season_df_filled.iloc[0:] = season_df.iloc[0]
/var/folders/8x/k_nyp_s16d10yl_nkhlgg7cc0000gn/T/ipykernel_70200/309712317.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'autumn' has dtype incompatible with float64, please explicitly cast to a comp

# Modeling and Prediction
TODO: Define the model and fit it using training data. Then, use test data to make predictions

In [ ]:
"""
This defines the model, fits training data and then does the prediction
with the test data 

Parameters
----------
X_train: matrix of floats, training input with 10 features
y_train: array of floats, training output
X_test: matrix of floats: dim = (100, ?), test input with 10 features

Compute
----------
y_test: array of floats: dim = (100,), predictions on test set
"""
class Model(object):
    def __init__(self):
        super().__init__()
        self._x_train = None
        self._y_train = None

    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        #TODO: Define the model and fit it using (X_train, y_train)
        self._x_train = X_train
        self._y_train = y_train

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        #TODO: Use the model to make predictions y_pred using test data X_test
        y_pred=np.zeros(X_test.shape[0])
        assert y_pred.shape == (X_test.shape[0],), "Invalid data shape"
        return y_pred

In [ ]:
model = Model()
# Use this function for training the model
model.train(X_train=X_train, y_train=y_train)
# Use this function for inferece
y_pred = model.predict(X_test)

# Saving Results
You don't have to change this

In [ ]:
dt = pd.DataFrame(y_pred) 
dt.columns = ['price_CHF']
dt.to_csv('results.csv', index=False)
print("\nResults file successfully generated!")